<a href="https://colab.research.google.com/github/esheshka/chess/blob/main/chess_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **Globals**

table = [['.' for i in range(8)] for i in range(8)]
white_figures = ['P', 'R', 'N', 'B', 'Q', 'K']
black_figures = ['p', 'r', 'n', 'b', 'q', 'k']
figures = [*white_figures, *black_figures]
white_move = True
move_count = 1

In [2]:
def create_table():
    global table
    global white_move
    global move_count

    table = [['.' for i in range(8)] for i in range(8)]

    table[0] = ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
    table[7] = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
    for x in range(8):
        table[1][x] = 'p'
        table[6][x] = 'P'

    table = flip(transpose(table))

    white_move = True
    move_count = 1

In [3]:
def transpose(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[i][j]]
        res = res + [tmp]

    return res

In [4]:
def flip(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[j][7 - i]]
        res = res + [tmp]

    return res

In [5]:
def draw():
    # for i in range(8):
    #     print(i, "-", *table[i])

    print(f"\n   A B C D E F G H\n")
    for x in range(8):
        print(8 - x, end='  ')

        for y in range(8):
            print(table[y][7 - x], end=' ')

        print('', 8 - x)

    print(f"\n   A B C D E F G H\n")

In [6]:
def coordinates(text):
    result = [ord(text[0]) - ord('a'), int(text[1]) - 1]

    return result

In [18]:
def move(command):
    if len(command) == 3 and command[0] in "abcdefghABSDEFGH" and command[1] in "12345678" and command[2] == "-":
        error = help(command)
        if error:
            return error
        return 'ok'

    if len(command) == 2 and command[0] in "abcdefghABSDEFGH" and command[1] in "12345678":
        error = help(command, attack=True)
        if error:
            return error
        return 'ok'

    if len(command) != 5 or command[0] not in "abcdefghABSDEFGH" or command[1] not in "12345678" or command[2] != "-" or command[3] not in "abcdefghABSDEFGH" or command[4] not in "12345678":
        return "Wrong input format."

    move_from, move_to = map(coordinates, command.split("-"))

    if table[move_from[0]][move_from[1]] == ".":
        return "The piece cannot make the specified move."
    elif white_move and table[move_from[0]][move_from[1]] in black_figures:
        return "The piece cannot make the specified move."
    elif (not white_move) and table[move_from[0]][move_from[1]] in white_figures:
        return "The piece cannot make the specified move."

    possible = possible_moves(move_from[0], move_from[1])
    if move_to not in possible:
        return "The piece cannot make the specified move."
    else:
        table[move_to[0]][move_to[1]] = table[move_from[0]][move_from[1]]
        table[move_from[0]][move_from[1]] = '.'

In [29]:
def help(command, attack=False):
    move_from = coordinates(command[:2])

    if table[move_from[0]][move_from[1]] == ".":
        if attack:
            print("enemy figures under attack: none")
        else:
            print("possible moves: none")
        return
    elif white_move and table[move_from[0]][move_from[1]] in black_figures:
        if attack:
            print("enemy figures under attack: none")
        else:
            print("possible moves: none")
        return
    elif (not white_move) and table[move_from[0]][move_from[1]] in white_figures:
        if attack:
            print("enemy figures under attack: none")
        else:
            print("possible moves: none")
        return

    possible = possible_moves(move_from[0], move_from[1])

    pos_coords = []
    for position in possible:
        if attack and table[position[0]][position[1]] not in figures:
            continue

        pos_coor = chr(ord('a') + position[0]) + str(position[1] + 1)
        pos_coords.append(pos_coor)

    pos_coords.sort()
    if attack:
        print("enemy figures under attack: ", end='')
    else:
        print("possible moves: ", end='')

    if len(pos_coords) == 0:
        print('none', end='')
    else:
        print(pos_coords[0], end='')
        for position in pos_coords[1:]:
            print(',', position, end='')

    print()

In [20]:
#@title **Возможные ходы**

def possible_moves(x, y):
    pos = []
    if table[x][y] in 'pP':
        pos = [*pos, *possible_moves_pawn(x, y)]
    if table[x][y] in 'rRqQ':
        pos = [*pos, *possible_moves_rook(x, y)]
    if table[x][y] in 'bBqQ':
        pos = [*pos, *possible_moves_bishop(x, y)]
    if table[x][y] in 'nN':
        pos = [*pos, *possible_moves_knight(x, y)]
    if table[x][y] in 'kK':
        pos = [*pos, *possible_moves_king(x, y)]


    return pos

In [21]:
#@title **Возможные ходы короля**

def possible_moves_king(x, y):
    pos = []
    for x1 in [-1, 0, 1]:
        for y1 in [-1, 0, 1]:
            if (x1 == 0 and y1 == 0) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [22]:
#@title **Возможные ходы коня**

def possible_moves_knight(x, y):
    pos = []
    for x1 in [-2, -1, 1, 2]:
        for y1 in [-2, -1, 1, 2]:
            if (x1*x1 + y1*y1 != 5) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [23]:
#@title **Возможные ходы слона**

def possible_moves_bishop(x, y):
    pos = []
    for a in range(1, 8):
        if x + a not in range(8) or y + a not in range(8):
            break
        elif table[x+a][y+a] == '.':
            pos.append([x+a, y+a])
        elif (table[x+a][y+a] in white_figures) == white_move:
            break
        else:
            pos.append([x+a, y+a])
            break

    for a in range(1, 8):
        if x + a not in range(8) or y - a not in range(8):
            break
        elif table[x+a][y-a] == '.':
            pos.append([x+a, y-a])
        elif (table[x+a][y-a] in white_figures) == white_move:
            break
        else:
            pos.append([x+a, y-a])
            break

    for a in range(1, 8):
        if x - a not in range(8) or y + a not in range(8):
            break
        elif table[x-a][y+a] == '.':
            pos.append([x-a, y+a])
        elif (table[x-a][y+a] in white_figures) == white_move:
            break
        else:
            pos.append([x-a, y+a])
            break

    for a in range(1, 8):
        if x - a not in range(8) or y - a not in range(8):
            break
        elif table[x-a][y-a] == '.':
            pos.append([x-a, y-a])
        elif (table[x-a][y-a] in white_figures) == white_move:
            break
        else:
            pos.append([x-a, y-a])
            break

    return pos

In [24]:
#@title **Возможные ходы ладьи**

def possible_moves_rook(x, y):
    pos = []
    for x1 in range(1, 8):
        if x + x1 not in range(8):
            break
        elif table[x+x1][y] == '.':
            pos.append([x+x1, y])
        elif (table[x+x1][y] in white_figures) == white_move:
            break
        else:
            pos.append([x+x1, y])
            break

    for x1 in range(1, 8):
        if x - x1 not in range(8):
            break
        elif table[x-x1][y] == '.':
            pos.append([x-x1, y])
        elif (table[x-x1][y] in white_figures) == white_move:
            break
        else:
            pos.append([x-x1, y])
            break

    for y1 in range(1, 8):
        if y + y1 not in range(8):
            break
        elif table[x][y+y1] == '.':
            pos.append([x, y+y1])
        elif (table[x][y+y1] in white_figures) == white_move:
            break
        else:
            pos.append([x, y+y1])
            break

    for y1 in range(1, 8):
        if y - y1 not in range(8):
            break
        elif table[x][y-y1] == '.':
            pos.append([x, y-y1])
        elif (table[x][y-y1] in white_figures) == white_move:
            break
        else:
            pos.append([x, y-y1])
            break

    return pos

In [25]:
#@title **Возможные ходы пешки**

def possible_moves_pawn(x, y):
    pos = []
    if white_move:
        if y+1 in range(8):
            if x+1 in range(8) and table[x+1][y+1] in black_figures:
                pos.append([x+1, y+1])
            if x-1 in range(8) and table[x-1][y+1] in black_figures:
                pos.append([x-1, y+1])

            if table[x][y+1] not in figures:
                pos.append([x, y+1])
                if y == 1 and table[x][y+2] not in figures:
                    pos.append([x, y+2])
    else:
        if y-1 in range(8):
            if x+1 in range(8) and table[x+1][y-1] in white_figures:
                pos.append([x+1, y-1])
            if x-1 in range(8) and table[x-1][y-1] in white_figures:
                pos.append([x-1, y-1])

            if table[x][y-1] not in figures:
                pos.append([x, y-1])
                if y == 6 and table[x][y-2] not in figures:
                    pos.append([x, y-2])

    return pos

In [26]:
def my_main(command):
    global white_move
    global move_count

    if command == 'exit':
        return True
    elif command == 'draw':
        draw()
    else:
        error = move(command)
        if not error:
            white_move = not white_move
            if white_move:
                move_count += 1
        elif error != "ok":
            print('Error. Type:', error)

In [30]:
create_table()

while True:
    print('white'*white_move + 'black'*(not white_move), str(move_count) + ':')
    if my_main(input()):
        break


white 1:
e3
enemy figures under attack: none
white 1:
e3-
possible moves: none
white 1:
e7
enemy figures under attack: none
white 1:
exit


In [ ]:
# create_table()

# moves = '''e2-e4
# e7-e5
# g1-f3
# b8-c6
# f1-c4
# g8-f6
# c2-c3
# d7-d6
# d4-d5
# f6-e4
# d5-e6
# d7-e6
# f3-g5
# d8-f6
# draw
# exit'''

# for com in moves.split("\n"):
#     print('white'*white_move + 'black'*(not white_move), str(move_count) + ':')
#     my_main(com)